<a href="https://colab.research.google.com/github/fllay/lab/blob/main/tensorflowjstest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi 

In [ ]:
!pip install flask
!pip install flask_cors
!apt-get install -qq protobuf-compiler python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive
!pip install -q pycocotools
!pip install tf_slim
!pip install tf-models-official
!pip install flask-executor

%mkdir /content/uploads
%mkdir /content/images
%mkdir /content/images/test
%mkdir /content/images/train
%mkdir /content/training
%mkdir /content/inference_graph


In [ ]:
!pip install tensorflowjs

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from google.protobuf import text_format
from object_detection.protos.string_int_label_map_pb2 import StringIntLabelMap, StringIntLabelMapItem
from object_detection.protos import pipeline_pb2

%cd /content/models/research
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import tensorflow as tf
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        #print(xml_file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            
            try:
              #print(member[0].text)
              value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
              xml_list.append(value)
            except IndexError:
              pass
    
            
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [ ]:
import tensorflow.compat.v1 as tf1
from collections import namedtuple, OrderedDict
import io
from PIL import Image
from object_detection.utils import dataset_util


def create_tf_example(group, path, label_dict):
    with tf1.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(label_dict[row['class']])

    tf_example = tf1.train.Example(features=tf1.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def split_f(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def gen_tfrecord(image_dir, csv_input, output_path, label_dict):
    writer = tf1.python_io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = pd.read_csv(csv_input)
    grouped = split_f(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path, label_dict)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))

def convert_classes(classes, start=1):
    msg = StringIntLabelMap()
    label_dict = {}
    for id, name in enumerate(classes, start=start):
      msg.item.append(StringIntLabelMapItem(id=id, name=name))
      label_dict[name] = id

    text = str(text_format.MessageToBytes(msg, as_utf8=True), 'utf-8')
    return text, label_dict

In [ ]:
%cd /content
!wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
!tar -xvf mobilenet_v2.tar.gz
!rm mobilenet_v2.tar.gz

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
!cp ssd_mobilenet_v2_320x320_coco17_tpu-8.config mobilenet_v2.config

In [ ]:
!wget https://raw.githubusercontent.com/hugozanini/object-detection/master/inferenceutils.py
from inferenceutils import *

In [ ]:
%cd /content
!git clone https://github.com/fllay/lab.git

In [ ]:
NUM_STEPS = 2000

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/training

In [ ]:
from flask import Flask, render_template, request, abort, jsonify, send_file
from flask_cors import CORS
import os
from os import path
from flask import Flask, flash, make_response, request, redirect, render_template
from werkzeug.utils import secure_filename
from flask_executor import Executor
import shutil
import zipfile
import glob
import random
import re
from multiprocessing import Queue

PATH = '/content'

TEMPLATE_DIR = os.path.abspath(PATH +'/lab/lab1')
UPLOAD_FOLDER = os.path.join(PATH, 'uploads')
IMAGE_FOLDER = os.path.join(PATH, 'images')
IMAGE_FOLDER_TEST = os.path.join(PATH, 'images', 'test')
IMAGE_FOLDER_TRAIN = os.path.join(PATH, 'images','train')

TEST_SIZE = 10

output_q = Queue()

def run(cmd):

  output = get_ipython().getoutput(cmd)
  print("###################")
  print(output)
  output_q.put(output)


app = Flask(__name__, template_folder=TEMPLATE_DIR)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
executor = Executor(app)
#CORS(app)


def train():
  model_dir = '/content/training/'
  labelmap_path = '/content/images/object-detection.pbtxt'
  pipeline_config_path = '/content/mobilenet_v2.config'
  fine_tune_checkpoint = '/content/mobilenet_v2/mobilenet_v2.ckpt-1'
  num_steps = NUM_STEPS
  num_eval_steps = 100
  run('python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=/content/mobilenet_v2.config \
    --model_dir=/content/training/ \
    --alsologtostderr \
    --num_train_steps=2000 \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps=100')
  return 0


def export_model():
  model_dir = '/content/training/'
  labelmap_path = '/content/images/object-detection.pbtxt'
  pipeline_config_path = '/content/mobilenet_v2.config'
  fine_tune_checkpoint = '/content/mobilenet_v2/mobilenet_v2.ckpt-1'
  num_steps = NUM_STEPS
  num_eval_steps = 100
  output_directory = '/content/inference_graph'
  run('python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir /content/training/ \
    --output_directory /content/inference_graph \
    --pipeline_config_path /content/mobilenet_v2.config')
  return 0

@app.route('/')
def index():
    return render_template('index.html')
    #return 'Web App with Python Flask!'

@app.route('/hello', methods=['GET'])
def hello():
    print("hello called")
    message = {
          'message': "Hello world222"
    }
    return jsonify(message)

@app.route('/trainObject', methods=['POST','PUT'])
def trainObject():
  executor.submit_stored('runcmd', train) 
  return make_response(("Finish training", 200))
 

@app.route('/exportModelObject', methods=['POST','PUT'])
def exportModelObject():
  executor.submit_stored('runcmd', export_model)
  return make_response(("Finish export", 200))

@app.route('/getResult',  methods=['POST','PUT'])
def get_result():
    if not executor.futures.done('runcmd'):
      if not output_q.empty():
        print('!!!!!!!!!!')
        print(output_q.get())
      return jsonify({'status': executor.futures._state('runcmd')})
    return jsonify({'status': "done"})

@app.route('/split', methods=['POST','PUT'])
def split_route():
    S_PATH = os.path.join(IMAGE_FOLDER, 'ttimage')
    print(S_PATH)
    files_xml=glob.glob(S_PATH+'/*.xml')
    files_png=[]
    file_xml_test=[]
    file_png_test=[]
    for i in range(TEST_SIZE):
      #files_png.append(os.path.splitext(f)[0]+".png")
      file_xml_test.append(files_xml.pop(random.randrange(len(files_xml))))
    print(files_xml)
    print(file_xml_test)
    for f in file_xml_test:
      png_file = os.path.splitext(f)[0]+".png"
      jpg_file = os.path.splitext(f)[0]+".jpg"
      if path.exists(png_file):
        file_png_test.append(png_file)
      elif path.exists(jpg_file):
        file_png_test.append(jpg_file)
    for f in files_xml:
      png_file = os.path.splitext(f)[0]+".png"
      jpg_file = os.path.splitext(f)[0]+".jpg"
      if path.exists(png_file):
        files_png.append(png_file)
      elif path.exists(jpg_file):
        files_png.append(jpg_file)
    print(files_png)
    print(file_png_test)
    for f in files_xml:
      fname = os.path.basename(f)
      dest = IMAGE_FOLDER_TRAIN +'/' + fname
      shutil.copyfile(f, dest)
    for f in files_png:
      fname = os.path.basename(f)
      dest = IMAGE_FOLDER_TRAIN +'/' + fname
      shutil.copyfile(f, dest)
    for f in file_xml_test:
      fname = os.path.basename(f)
      dest = IMAGE_FOLDER_TEST +'/' + fname
      shutil.copyfile(f, dest)
    for f in file_png_test:
      fname = os.path.basename(f)
      dest = IMAGE_FOLDER_TEST +'/' + fname
      shutil.copyfile(f, dest)

    xml_df=xml_to_csv(IMAGE_FOLDER_TRAIN)
    print(IMAGE_FOLDER + '/train_labels.csv')
    xml_df.to_csv(IMAGE_FOLDER + '/train_labels.csv', index=None)
    xml_df=xml_to_csv(IMAGE_FOLDER_TEST)
    print(IMAGE_FOLDER + '/train_labels.csv')
    xml_df.to_csv(IMAGE_FOLDER + '/test_labels.csv', index=None)  

    csv_file_train = pd.read_csv(IMAGE_FOLDER + '/train_labels.csv') 
    column_val_list_train = csv_file_train['class']
    all_labels_train = set(column_val_list_train) 

    csv_file_test = pd.read_csv(IMAGE_FOLDER + '/test_labels.csv') 
    column_val_list_test = csv_file_test['class']
    all_labels_test = set(column_val_list_test) 

    all_labels = all_labels_train.union(all_labels_test)
    print("All labels = "+ str(all_labels))

    s_labels = list(all_labels)
    print("All classes = "+ str(s_labels))

    NUM_OF_CLASS = len(s_labels)
    print(NUM_OF_CLASS)
    txt, label_dict = convert_classes(s_labels)
    print(txt)

    with open(IMAGE_FOLDER + '/object-detection.pbtxt', 'w') as f:
      f.write(txt)

    gen_tfrecord(image_dir=IMAGE_FOLDER_TRAIN, csv_input=IMAGE_FOLDER + '/train_labels.csv', output_path=IMAGE_FOLDER +'/train.record', label_dict=label_dict)
    gen_tfrecord(image_dir=IMAGE_FOLDER_TEST, csv_input=IMAGE_FOLDER + '/test_labels.csv', output_path=IMAGE_FOLDER + '/test.record', label_dict=label_dict)

    
    num_classes = NUM_OF_CLASS
    batch_size = 96
    num_steps = NUM_STEPS
    num_eval_steps = 100

    train_record_path = IMAGE_FOLDER +'/train.record'
    test_record_path = IMAGE_FOLDER + '/test.record'
    model_dir = '/content/training/'
    labelmap_path = '/content/images/object-detection.pbtxt'

    pipeline_config_path = '/content/mobilenet_v2.config'
    fine_tune_checkpoint = '/content/mobilenet_v2/mobilenet_v2.ckpt-1'

    filenameOut = pipeline_config_path
    filename = '/content/ssd_mobilenet_v2_320x320_coco17_tpu-8.config'
    pipeline_config = pipeline_pb2.TrainEvalPipelineConfig() 

    
    '''with tf.gfile.GFile(filename, "r") as f:                                                                                                                                                                                                                     
      proto_str = f.read()                                                                                                                                                                                                                                          
      text_format.Merge(proto_str, pipeline_config)  

    pipeline_config.model.ssd.num_classes = NUM_OF_CLASS
    pipeline_config.train_config.num_steps = num_steps
    pipeline_config.train_config.fine_tune_checkpoint=fine_tune_checkpoint
    pipeline_config.train_input_reader.label_map_path=labelmap_path
    pipeline_config.train_input_reader.tf_record_input_reader.input_path[0]=train_record_path
    pipeline_config.eval_input_reader[0].label_map_path=labelmap_path
    pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0]=test_record_path 
    pipeline_config.train_config.batch_size=batch_size

    config_text = text_format.MessageToString(pipeline_config)   
                                                                                                                                                                                                    
    with tf.gfile.Open(filenameOut, "wb") as f:                                                                                                                                                                                                                       
      f.write(config_text)   '''

    with open(pipeline_config_path) as f:
      config = f.read()

    with open(pipeline_config_path, 'w') as f:
      # Set labelmap path
      config = re.sub('label_map_path: ".*?"', 'label_map_path: "{}"'.format(labelmap_path), config)
      # Set fine_tune_checkpoint path
      config = re.sub('fine_tune_checkpoint: ".*?"', 'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
      # Set train tf-record file path
      config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_path), config)
      # Set test tf-record file path
      config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_path), config)
      # Set number of classes.
      config = re.sub('num_classes: [0-9]+', 'num_classes: {}'.format(num_classes), config)
      # Set batch size
      config = re.sub('batch_size: [0-9]+', 'batch_size: {}'.format(batch_size), config)
      # Set training steps
      config = re.sub('num_steps: [0-9]+', 'num_steps: {}'.format(num_steps), config)
      f.write(config)

    return make_response(("Got it", 200))

@app.route('/upload', methods=['POST','PUT'])
def upload():
    file = request.files['file']

    save_path = os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(file.filename))
    current_chunk = int(request.form['dzchunkindex'])
    print(save_path)

    # If the file already exists it's ok if we are appending to it,
    # but not if it's new file that would overwrite the existing one
    if os.path.exists(save_path) and current_chunk == 0:
        # 400 and 500s will tell dropzone that an error occurred and show an error
        
        print(current_chunk)
        return make_response(('File already exists', 400))

    try:
        with open(save_path, 'ab') as f:
            f.seek(int(request.form['dzchunkbyteoffset']))
            f.write(file.stream.read())
    except OSError:
        # log.exception will include the traceback so we can see what's wrong 
        print('Could not write to file')
        return make_response(("Not sure why,"
                              " but we couldn't write the file to disk", 500))

    total_chunks = int(request.form['dztotalchunkcount'])

    if current_chunk + 1 == total_chunks:
        # This was the last chunk, the file should be complete and the size we expect
        if os.path.getsize(save_path) != int(request.form['dztotalfilesize']):
            print(f"File {file.filename} was completed, "
                      f"but has a size mismatch."
                      f"Was {os.path.getsize(save_path)} but we"
                      f" expected {request.form['dztotalfilesize']} ")
            return make_response(('Size mismatch', 500))
        else:
            print(f'File {file.filename} has been uploaded successfully')
            with zipfile.ZipFile(save_path, 'r') as zip_ref:
              zip_ref.extractall('/content/images')
            
    else:
        print(f'Chunk {current_chunk + 1} of {total_chunks} '
                  f'for file {file.filename} complete')
        
        
        
    res1= jsonify({"status":"success","info":"com1","file_link":1})
    print(res1)
    return make_response((res1, 200))




from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))
app.run(host='0.0.0.0', port=5000)




#Train model

In [ ]:
!pip install flask-executor

In [ ]:
  run('python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=/content/mobilenet_v2.config \
    --model_dir=/content/training/ \
    --alsologtostderr \
    --num_train_steps=2000 \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps=100')

In [ ]:
model_dir = '/content/training/'
labelmap_path = '/content/images/object-detection.pbtxt'
pipeline_config_path = '/content/mobilenet_v2.config'
fine_tune_checkpoint = '/content/mobilenet_v2/mobilenet_v2.ckpt-1'
num_steps = NUM_STEPS
num_eval_steps = 100

In [ ]:


!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir}

In [ ]:
output_directory = '/content/inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

In [ ]:
!zip -r /content/saved_model.zip /content/inference_graph/saved_model/

In [ ]:
from google.colab import files
files.download("/content/saved_model.zip")

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)
tf.keras.backend.clear_session()
model = tf.saved_model.load(f'{output_directory}/saved_model')
import pandas as pd
test = pd.read_csv('/content/images/test_labels.csv')
#Getting 3 random images to test
images = list(test.sample(n=3)['filename'])
print(images)

for image_name in images:
  
  image_np = load_image_into_numpy_array('/content/images/test/' + image_name)
  output_dict = run_inference_for_single_image(model, image_np)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)
  display(Image.fromarray(image_np))

In [ ]:
!rm -rf /content/images/test
!rm -rf /content/images/train
!rm /content/images/*.csv
!rm /content/images/*.pbtxt
!rm /content/images/*.record
!mkdir /content/images/test
!mkdir /content/images/train
!mkdir /content/inference_graph
!!cp /content/ssd_mobilenet_v2_320x320_coco17_tpu-8.config /content/mobilenet_v2.config

In [ ]:
%mkdir /content/training

In [ ]:
%mkdir /content/inference_graph

In [ ]:
!pip install tensorflowjs[wizard]

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_format=tfjs_graph_model \
    --signature_name=serving_default \
    --saved_model_tags=serve \
    /content/inference_graph/saved_model \
    /content/web_model

In [ ]:
imported = tf.saved_model.load('/content/inference_graph/saved_model')

In [ ]:
print(imported)

In [ ]:
!zip -r /content/web_model.zip /content/web_model/
from google.colab import files
files.download("/content/web_model.zip")

# Exercise



*   Return tensorflow version in route `/hello`
*   Flask is a web application framework. Please try to find a way to write a POST method and write an API to add two number. You can test the API using `axois` in the html file. Please test the API in index.html file. 

